Convert to MusicXML and MIDI

In [21]:
import music21 as m21
import pandas as pd
import os

def convert_song_to_midi(song : m21.stream.Score):
    sXML = m21.stream.Stream()
    sMIDI = m21.stream.Stream()
    for part_id, part in enumerate(song.parts):
        print(part_id)
        sXML.append(part)
        p = m21.stream.Part()
        p.insert(m21.instrument.Bagpipes())
        print(part.partName)
        for event in part.flat.notesAndRests:
            if isinstance(event, m21.note.Note):
                print(event.pitch.midi, end=' ')
                print(event.duration.quarterLength, end='|')
                p.append(m21.note.Note(int(event.pitch.midi), quarterLength=event.duration.quarterLength))
            else:
                print("r", end='')
                print(event.duration.quarterLength)
                p.append(m21.note.Rest(quarterLength=event.duration.quarterLength))
        sMIDI.append(p)
        
    sXML.write('musicxml', f'{song.metadata.bestTitle}')
    sMIDI.write('midi', f'{song.metadata.bestTitle}.mid')
    
paths = m21.corpus.getComposer('bach')
songs = []
# for path in paths[0]:
song = m21.corpus.parse(paths[5])
songs.append(song)
convert_song_to_midi(songs[0])

0
Soprano
69 0.5|71 0.5|73 1.0|74 1.0|76 1.0|74 1.0|73 1.0|71 1.0|73 1.0|73 1.0|73 1.0|71 0.5|73 0.5|74 0.5|73 0.5|71 1.0|69 1.0|71 1.0|69 1.0|69 1.0|71 1.0|73 1.0|74 1.0|73 1.0|71 1.0|73 1.0|71 1.0|71 1.0|73 1.0|74 1.0|76 1.0|74 1.0|73 1.0|71 1.0|73 1.0|73 1.0|73 1.0|71 0.5|73 0.5|74 0.5|73 0.5|71 1.0|69 0.5|66 0.5|68 1.0|69 1.0|1
Alto
64 1.0|69 1.0|69 1.0|68 1.0|66 0.5|68 0.5|69 1.0|68 1.0|69 1.0|64 1.0|66 1.0|66 0.5|64 0.5|62 1.0|64 1.0|64 0.5|69 1.0|68 0.5|64 1.0|64 1.0|64 1.0|64 1.0|62 0.5|64 0.5|66 1.0|66 0.5|64 0.5|64 0.5|62 0.25|61 0.25|62 1.0|64 1.0|64 1.0|66 1.0|68 1.0|66 1.0|64 1.0|62 0.5|66 0.25|65 0.25|66 1.0|64 1.0|66 1.0|66 0.5|64 0.5|62 1.0|64 1.0|64 0.5|66 0.5|64 1.0|64 1.0|2
Tenor
61 0.5|62 0.5|64 1.0|66 1.0|59 0.5|61 0.5|62 1.0|64 1.0|64 1.0|64 1.0|57 1.0|57 3.0|62 0.5|59 0.5|64 0.5|62 0.25|61 0.25|62 1.0|61 1.0|61 1.0|56 1.0|58 1.0|59 1.0|58 1.0|59 1.0|58 1.0|54 1.0|59 1.0|57 1.0|57 1.0|59 1.0|57 0.5|59 0.5|61 1.0|54 0.5|56 0.5|58 1.0|57 1.0|57 2.0|59 1.0|56 1.0|57 

In [20]:
part_indexes = {
    0: 'soprano',
    1: 'alto',
    2: 'tenor',
    3: 'bass'    
}

def make_measure_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    return [lst[i:i+n] for i in range(len(lst)-n+1)]

def to_dataset(song:m21.stream.Score):
    with open('dataset.csv', 'w') as fd:
        fd.write('sixteenth,soprano,alto,tenor,bass,file\n')
        print(f'Processing {song.metadata.bestTitle}...')
        parts = {}
        # iterate over the 4 parts
        for part_id, part in enumerate(song.parts):
            # Get all the measures in this part
            measures = [mm for mm in part.getElementsByClass('Measure') if mm.number != 0]
            # Group them into groups of 4
            measure_chunks = make_measure_chunks(measures, 4)
            part_chunks = []
            for chunk in measure_chunks:
                # For every chunk of 4 measures
                chunk_encoding = {(offset/4.0): '--' for offset in range(0, 64)}
                for measure_id, measure in enumerate(chunk):
                    # Iterate over each measure
                    for ev in measure:
                        # And every note within the measure
                        offs = 4.0 * measure_id + ev.offset
                        if isinstance(ev, m21.chord.Chord):                        
                            if offs in chunk_encoding:
                                chunk_encoding[offs] = ev[0].nameWithOctave
                        elif isinstance(ev, m21.note.Note):
                            if offs in chunk_encoding:
                                chunk_encoding[offs] = ev.nameWithOctave                        
                        elif isinstance(ev, m21.note.Rest):                        
                            if offs in chunk_encoding:
                                chunk_encoding[offs] = 'Rest'
                                
                part_chunks.append(list(chunk_encoding.values()))
            parts[part_indexes[part_id]] = part_chunks
            
        for chunk_id in range(len(parts['soprano'])):
            dfdict = {}
            for part, chunks in parts.items():
                dfdict[part] = chunks[chunk_id]
            chunk_name = f'{song.metadata.bestTitle}_chunk_{chunk_id}'
            dfdict['file'] = chunk_name            
            df = pd.DataFrame(dfdict)
            df.to_csv(fd, header=False)
    print('The dataset.csv file has been written!')
    
paths = m21.corpus.getComposer('bach')
songs = []
# for path in paths[0]:
song = m21.corpus.parse(paths[5])
songs.append(song)
to_dataset(songs[0])

Processing bwv104.6.mxl...
The dataset.csv file has been written!
